Caixeiro viajante
=================



## Introdução



Até agora, sempre que nós aplicamos os operadores de `cruzamento` ou de `mutação` nós não nos preocupamos se o indivíduo gerado por estes processo era um `indivíduo válido`. Um indivíduo válido é aquele que representa uma solução possível e bem formulada para o problema em questão.

Por exemplo, no problema das caixas binárias, [1, 0, 0, 1] é um indivíduo válido para o caso de termos 4 caixas. Um exemplo de `indivíduo inválido` para este mesmo problema seria [1, 0, 0, a], pois um dos genes está assumindo um valor fora do domínio. Outro exemplo de indivíduo inválido poderia ser [1, 1, 0], pois é um indivíduo com apenas 3 genes, sendo que o esperado eram 4 genes.

Neste experimento nós veremos estratégias para evitar que indivíduos inválidos sejam obtidos quando usamos os operadores de cruzamento e de mutação. No notebook seguinte veremos como aplicar uma penalidade para indivíduos inválidos que forem gerados durante uma busca genética com restrições.



## Objetivo



Encontrar uma solução para o problema do caixeiro viajante. Considere que ele irá visitar 5 cidades, pode iniciar sua viagem por qualquer uma destas cidades e deve retornar à cidade de início. Durante seu trajeto, não pode visitar a mesma cidade duas vezes (única exceção é a cidade inicial).



## Descrição do problema



O problema consiste em descobrir a rota de menor distância entre $n$ pontos no plano cartesiano (ou seja, $n$ pontos com coordenadas $(x,y)$). A rota pode se iniciar em qualquer um dos pontos disponíveis e deve terminar no ponto inicial, visitando todos os demais pontos apenas uma vez. Considere que a rota entre um ponto e outro é a linha reta que liga os dois pontos.

O código abaixo é um gerador de coordenadas de cidades. Use ele caso queira.



## Importações



<p style='text-align: justify'>
<ul>
    <li> Importando 'random', a função permutations de itertools, e algumas funções criadas, do arquivo 'funcoes.py', a serem usadas no algoritmo e as renomeando de maneira "auto-explicativa":</li>
</ul></p>

In [1]:
import random
from itertools import permutations

from funcoes import cria_cidades

from funcoes import populacao_inicial_cv as cria_populacao_inicial
from funcoes import funcao_objetivo_pop_cv
from funcoes import funcao_objetivo_cv
from funcoes import selecao_torneio_min as funcao_selecao
from funcoes import cruzamento_ordenado as funcao_cruzamento
from funcoes import mutacao_de_troca as funcao_mutacao

## Códigos e discussão



<p style='text-align: justify'>
<ul>
    <li>Definindo as constantes, sendo elas: relacionadas à busca, que, se mudadas, alteram a eficácia do algoritmo; e relacionadas ao problema a ser resolvido, que, caso alteradas, mudam o problema em questão que se está resolvendo:</li>
</ul></p>

In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 50
NUM_GERACOES = 1000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3

# relacionadas ao problema a ser resolvido
NUMERO_DE_CIDADES = 5
CIDADES = cria_cidades(NUMERO_DE_CIDADES)

<p style='text-align: justify'>
<ul>
    <li> Definindo as funções locais, as quais utilizam de alguns valores padrões definidos neste notebook:</li>
</ul></p>

In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_cv(populacao, CIDADES)

def funcao_objetivo_individuo(individuo):
    return funcao_objetivo_cv(individuo, CIDADES)

<p style='text-align: justify'>
<ul>
    <li> Código contendo a lógica para a resolução do problema:</li>
</ul></p>

In [4]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, CIDADES)

melhor_fitness_ja_visto = float("inf")  # escrevendo infinito em python, já que claramente poderá ser
                                        # substituido por um menor valor encontrado

for n in range(NUM_GERACOES):
    
    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]
    
    contador = 0
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
        
        contador = contador + 2   
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)            
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    if menor_fitness < melhor_fitness_ja_visto:        
        posicao = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = menor_fitness 

In [5]:
# Busca testando todas as permutações

melhor_fitness_ever = float("inf")

# testando todas as permutações possíveis e buscando o menor fitness dentre elas
for caminho in permutations(list(CIDADES.keys())):
    distancia = funcao_objetivo_individuo(caminho)
    if distancia < melhor_fitness_ever:
        melhor_fitness_ever = distancia
        melhor_resposta_ever = caminho

In [6]:
# Checando os resultados

print()
print("Melhor individuo obtido por algoritmos genéticos:")
print(melhor_individuo_ja_visto, "com distância:", melhor_fitness_ja_visto)

print()
print("Melhor individuo obtido por busca exaustiva:")
print(melhor_resposta_ever, "com distância:", melhor_fitness_ever)


Melhor individuo obtido por algoritmos genéticos:
['Cidade 4', 'Cidade 3', 'Cidade 0', 'Cidade 1', 'Cidade 2'] com distância: 2.2849728904112663

Melhor individuo obtido por busca exaustiva:
('Cidade 0', 'Cidade 1', 'Cidade 3', 'Cidade 2', 'Cidade 4') com distância: 2.2849728904112663


## Conclusão

<p style='text-align: justify'> Com este algoritmo algoritmo genético, foi possível resolver o problema do caixeiro viajante que busca sempre fazer o menor caminho possível, mas tendo em vista que ele é um problema do tipo NP difícil. Logo, não há um algoritmo eficiente que encontra a melhor/menor rota para o caixeiro... aqui, conseguimos encontrar uma que aparenta ter a mesma distância do que a encontrada por busca exaustiva (testando uma a uma), pelo menos no número de casas visíveis. Para isso: foram utilizadas novas funções para criar a população - onde, agora, é um conjunto de indivíduos que são possíveis caminhos pelas cidades que o caixeiro passou, indicando suas coordenadas - e para ser a objetivo - que precisa retornar a distância de todo o trajeto percorrido por ele; como seleção, foi utilizada a função de torneio mínimo e também foram usadas as funções de cruzamento ordenado e mutação de troca. </p>
    
<p style='text-align: justify'> Assim como no <a href="https://github.com/viyuetuki/aula_redes/blob/main/AlgoritmosGeneticos/experimento%20A.05%20-%20descobrindo%20a%20senha.ipynb">anterior</a>, de descobrir a senha, se caracteriza como um problema onde se busca a minimização da fitness.</p>

<p style='text-align: justify'> Este algoritmo se configura como <b>probabilístico</b>, já que seu resultado muda a cada vez que é rodado, fornecendo resultados diversos para pessoas diferentes também. Enquanto isso, a busca feita pelo modo exaustivo, realizando permutações, se caracteriza como determinística, já que testa uma a uma as possibilidades, fornecendo apenas uma resposta ao final.</p>

<p style='text-align: justify'> Por ser um problema NP difícil - que, inclusive, são os grandes problemas atuais para se resolver -, não há um algoritmo totalmente eficiente para resolver. Aqui, foi interessante que conseguimos testar, para memsos conjuntos de cidades criados a cada tentativa, tanto o algoritmo quanto a busca exaustiva (o que nem sempre é possível de se fazer, posto a complexidade). Isso colocou à vista que obtivemos, para os dois modos de resolução, trajétos diferentes, mas de mesma distância, o que nos faz questionar o porquê disso. Nesse sentido, podemos pensar que que isso ocorreu, porque, possivelmente, para o número de casas decimais sendo mostradas isos ocorre, mas há sim diferença entre elas mais adiante. Logo, não necessariamente a resposta do algoritmo é a melhor dentre todas as possíveis. </p>

## Playground

